In [1]:
import os
import pdfplumber
from elasticsearch import Elasticsearch
import urllib3
import re

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Connect to Elasticsearch
es = Elasticsearch(['http://localhost:9200'], verify_certs=False)

# Specify the folder path containing the resumes
folder_path = '/Users/krishnaprasath/Documents/Resumes'

# Index the resumes in the folder
index_name = 'resumes'  # Specify the index name

# Counter for indexing
document_count = 0

# Define different name patterns to capture
name_patterns = [
    r"(?i)^\s*(?:Name:|)(.*)",  # Name: John Doe
    r"(?i)^\s*(?:Mr\.|Ms\.|Mrs\.)(?:\.\s*)?(.*?)",  # Mr./Ms./Mrs. John Doe
    r"(?i)^\s*(?:First|Given)\s*Name:\s*(.*?)",  # First/Given Name: John
    r"(?i)^\s*(?:Last|Family)\s*Name:\s*(.*?)",  # Last/Family Name: Doe
    r"(?i)^\s*(?:Full|)(?:\s*Name:|)\s*(.*?)$"  # Full Name: John Doe
]

# Define pattern for mobile number extraction
mobile_pattern = r"\b(\d{10})\b"

# Define pattern for email extraction
email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.pdf'):
        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)

        # Open the PDF file using pdfplumber
        with pdfplumber.open(file_path) as pdf:
            # Extract text from the PDF
            resume_text = ""
            for page in pdf.pages:
                resume_text += page.extract_text()

        # Extract name using the defined patterns
        name = ""
        for pattern in name_patterns:
            name_match = re.search(pattern, resume_text, re.MULTILINE)
            if name_match:
                name = name_match.group(1).strip()
                break

        # Extract mobile number using the mobile pattern
        mobile = ""
        mobile_match = re.findall(mobile_pattern, resume_text, re.MULTILINE)
        if mobile_match:
            mobile = ", ".join(set(mobile_match))  # Replace all duplicates with a single instance

        # Extract email using the email pattern
        email = ""
        email_match = re.findall(email_pattern, resume_text, re.MULTILINE)
        if email_match:
            email = ", ".join(set(email_match))  # Replace all duplicates with a single instance

        # Create a dictionary with the extracted data
        resume_data = {}
        if name:
            resume_data['name'] = name
        if mobile:
            resume_data['mobile'] = mobile
        if email:
            resume_data['email'] = email

        # Index the resume data into Elasticsearch
        if resume_data:
            document_count += 1
            document_id = f"resume-{document_count}"  # Generate a unique document ID

            # Index the document into Elasticsearch
            resp = es.index(index=index_name, id=document_id, body=resume_data)
            if resp['result'] == 'created':
                print(f"Indexed {file_name} successfully.")
            else:
                print(f"Failed to index {file_name}.")
        else:
            print(f"No relevant information found in {file_name}.")

        print()

# Retrieve the indexed documents from Elasticsearch
resp = es.search(index=index_name, q='*')
hits = resp['hits']['hits']
print(f"Total indexed documents: {len(hits)}")
for hit in hits:
    print(f"Indexed data for {hit['_id']}:")
    print(hit['_source'])
    print()


/var/folders/vn/j8tq_4qj7k3107r087mbg3w00000gp/T/ipykernel_4184/1052391509.py:84: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  resp = es.index(index=index_name, id=document_id, body=resume_data)
/var/folders/vn/j8tq_4qj7k3107r087mbg3w00000gp/T/ipykernel_4184/1052391509.py:84: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  resp = es.index(index=index_name, id=document_id, body=resume_data)


Failed to index DuraimuruganCK.pdf.

Failed to index resume Sanjay.pdf.

Failed to index Krishna Prasath Resume (2).pdf.

Total indexed documents: 4
Indexed data for 1:
{'name': 'PROJECT', 'email': 'Hacker Earth: https://www.hackerearth.com/@krishna2828Communication skills', 'skills': []}

Indexed data for resume-1:
{'name': 'S', 'mobile': '8883157450', 'email': 'mdurai688@gmail.com'}

Indexed data for resume-2:
{'name': 'SANJAI T', 'mobile': '9994345779', 'email': 'rssanjai2002@gmail.com'}

Indexed data for resume-3:
{'name': 'KRISHNA', 'mobile': '9944614475', 'email': 'krishnaprasath0408@gmail.com'}



/var/folders/vn/j8tq_4qj7k3107r087mbg3w00000gp/T/ipykernel_4184/1052391509.py:95: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  resp = es.search(index=index_name, q='*')
